In [9]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_curve, auc, roc_auc_score
import lightgbm as lgb
from lightgbm import LGBMClassifier
import gc
from sklearn.grid_search import GridSearchCV
import time
import warnings
warnings.filterwarnings('ignore')



In [2]:
# Load Data

file_name = "../data/train_preprocessed2.csv"
train_df = pd.read_csv(file_name, low_memory = False)

train_df.head()

,A..papers,A.papers,B.papers,C.papers,Dif.countries,Perc_non_australian,Number.people,PHD,Max.years.univ,Grants.succ,...,SEO.11,SEO.12,SEO.13,SEO.14,SEO.15,SEO.16,SEO.17,SEO.18,SEO.19,Grant.Status
0,4.0,2.0,0.0,0.0,1,0.00,1,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,6.0,12.0,2.0,2.0,1,1.00,1,1.0,20.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,7.0,20.0,20.0,7.0,2,0.75,4,2.0,50.0,0.0,...,0,0,2,0,0,0,0,0,0,1
3,0.0,3.0,13.0,3.0,1,1.00,2,2.0,15.0,0.0,...,0,0,2,0,0,0,0,0,0,1
4,3.0,0.0,1.0,0.0,1,0.00,1,1.0,10.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [3]:
#Setup data : Divide Test and Train set

array = train_df.values

data = array[:, 0:70]
target = array[:, 70]

data, target

seed = 7
test_size = 0.2

data_train, data_test, target_train, target_test = train_test_split(data, target, test_size = test_size, random_state = seed)



In [4]:
# XGB (parameter default) Result 

model = xgb.XGBClassifier(eval_metric = 'auc')

# make predictions with kfold cross validation score
kfold = KFold(n_splits = 5, random_state = 7)
results = cross_val_score(model, data, target, cv = kfold)
accuracy = results.mean()*100
print("Accuracy : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    

Accuracy : 84.78% (2.85%)


In [5]:
# XGB model Function using KFold 

def XGB_Train_Model_using_KFold(min_child_weight, max_depth, gamma, subsample,  colsample_bytree) : 
    xgb_params = {
        'n_trees' : 20,
        'eta' : 0.2,
        'max_depth' : int(max_depth),
        'subsample' : max(min(subsample, 1), 0),
        'objective' : 'reg:linear', 
        'eval_metric' : 'auc',
        'silent' : 1,
        'min_child_weight' : int(min_child_weight),
        'gamma' : max(gamma, 0), 
        'colsample_bytree' : max(min(colsample_bytree, 1), 0)
    }
    
    model = xgb.XGBClassifier(**xgb_params)
    
    kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
    results = cross_val_score(model, data, target, cv = kfold)
    auc = results.mean()*100
    print("AUC : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    return auc

In [8]:
# XGB model Function using .cv

def XGB_Train_Model_using_cv(min_child_weight, max_depth, gamma, subsample,  colsample_bytree) : 
    xgb_params = {
        'n_trees' : 20,
        'eta' : 0.2,
        'max_depth' : int(max_depth),
        'subsample' : max(min(subsample, 1), 0),
        'objective' : 'reg:linear', 
        'eval_metric' : 'auc',
        'silent' : 1,
        'min_child_weight' : int(min_child_weight),
        'gamma' : max(gamma, 0), 
        'colsample_bytree' : max(min(colsample_bytree, 1), 0)
    }
    #TODO : dtrain값을 setting하는 법 연구!
    
    xgb_data = xgb.DMatrix(file_name)
    
    cv_results = xgb.cv(params = xgb_params, dtrain = xgb_data, num_boost_round = 20, nfold = 5, metrics = 'auc', early_stopping_rounds = 5, seed = 7, shuffle = True)
    #cv_results = lgb.cv(lgb_params, lgb_train, num_boost_round=int(n_boost_roundP), nfold=5, early_stopping_rounds=5)
    return cv_results['auc-mean'][-1]
    

In [6]:
#XGB (parameters tuned by Bayesian Optimization) result

xgb_params = {
    
    #Learning Rate 
    #'eta' : (0.01, 0.2),
    
    #Minimum sum of weights : to control overfitting
    'min_child_weight' : (1, 20), 
    
    #Maximum depth of a tree : to control overfitting
    'max_depth' : (2, 10),
    
    #minimum loss reduction required to make a split : makes algorithm conservative
    'gamma' : (0, 10), 
    
    #max_delta_step is not needed since data is not imbalanced
    #'max_delta_step' : (0, 10),
    
    #Fraction of observations to be randomly samples for each tree
    #Lower: prevent overfitting
    'subsample' : (0.5, 1),
    
    #Fraction of columns to be randomly samples for each tree
    'colsample_bytree' : (0.1, 1),
    
    #colsamble_bylevel is not needed since subsample and colsample_bytree will do the job
    #'colsample_bylevel' = (0.1, 1),
    
    #L2 regularization term on weights
    #'lambda' = (?, ?)
    
    #L1 regularization term on weight
    #'alpha' = (?, ?)
    
    #scale_pos_weight is not needed since data is not imbalanced
    #'scale_pos_weight' = (0, 10)
    #'n_splits_param' : (5, 10)
}


xgb_bayesOPT = BayesianOptimization(XGB_Train_Model_using_KFold, xgb_params)
start_time = time.time()
xgb_bayesOPT.maximize(init_points = 5, n_iter = 100)
elapsed_time = time.time() - start_time
print("elapsed time : %s"%elapsed_time)

Initialization
---------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
AUC : 85.15% (0.75%)
    1 | 00m02s |   85.15161 |             0.9823 |    6.6780 |      2.3374 |            13.0784 |      0.9223 | 
AUC : 86.67% (1.04%)
    2 | 00m06s |   86.66751 |             0.9430 |    2.9550 |      4.7989 |            18.4037 |      0.5374 | 
AUC : 85.47% (1.17%)
    3 | 00m01s |   85.47325 |             0.2415 |    9.5343 |      3.1746 |            18.1711 |      0.5804 | 
AUC : 86.91% (0.93%)
    4 | 00m07s |   86.90865 |             0.8571 |    6.7538 |      7.5942 |            17.3524 |      0.9311 | 
AUC : 86.01% (1.03%)
    5 | 00m06s |   86.01293 |             0.9699 |    7.4696 |      5.1763 |             6.7275 |      0.7492 | 
Bayesian Optimization
--------------------------------------------------------------------

AUC : 87.83% (0.90%)
   54 | 00m12s |   87.82742 |             0.4092 |    3.0584 |      5.1446 |            15.2599 |      1.0000 | 
AUC : 87.03% (0.91%)
   55 | 00m12s |   87.03499 |             0.5116 |    5.1229 |      4.0975 |            13.9949 |      0.7474 | 
AUC : 87.99% (0.80%)
   56 | 00m14s |   87.98813 |             0.3456 |    3.9437 |      6.5053 |            13.7876 |      0.8638 | 
AUC : 85.66% (1.23%)
   57 | 00m11s |   85.65711 |             0.1000 |    3.9004 |      4.9290 |            14.5270 |      0.9735 | 
AUC : 86.12% (1.04%)
   58 | 00m15s |   86.11637 |             0.9470 |    3.9574 |      3.7992 |            14.6991 |      0.6399 | 
AUC : 87.41% (0.94%)
   59 | 00m17s |   87.41389 |             0.7052 |    4.2789 |      7.3945 |            13.5479 |      0.7706 | 
AUC : 85.77% (1.04%)
   60 | 00m13s |   85.77185 |             0.9737 |    6.0597 |      3.6692 |            12.9919 |      0.8993 | 
AUC : 87.18% (0.56%)
   61 | 00m11s |   87.18419 |            

In [7]:
#lightGBM (parameter default) result

lgb_train = lgb.Dataset(data, target)
lgb_params = {
        
    #static parameters
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'feature_fraction': 0.9,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'verbose': 0
}
cv_results = lgb.cv(lgb_params, lgb_train, num_boost_round=20, nfold=5, stratified = False, 
                verbose_eval=20, early_stopping_rounds=5)
cv_results['auc-mean'][-1]

[20]	cv_agg's auc: 0.952699 + 0.00317399	cv_agg's l2: 0.0911689 + 0.00205443


0.9526990791573944

In [8]:
#LGB model function (eval metric = auc, using kfold)

def LGB_Train_Model(gamma, max_depth, min_child_weight, colsample_bytree, subsample) :
    lgb_train = lgb.Dataset(data, target)
    
    # specify your configurations as a dict
    lgb_params = {
        
    #static parameters
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'feature_fraction': 0.9,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'verbose': 0,
    'learning_rate': 0.2,
        
    'num_iterations' : int(num_iterations),
    'max_depth': int(max_depth),
    'min_child_weight' : int(min_child_weight),
    'colsample_bytree' : max(min(colsample_bytree, 1), 0),
    'subsample' : max(min(subsample, 1), 0),
    'gamma' : max(gamma, 0), 
    }

    cv_results = lgb.cv(lgb_params, lgb_train, num_boost_round=20, nfold=5, early_stopping_rounds=5, stratified = False)
    return cv_results['auc-mean'][-1]
    

In [11]:
def LGB_Train_Model_using_cv(gamma, max_depth, min_child_weight, colsample_bytree, subsample) :
    lgb_train = lgb.Dataset(data, target)
    
    # specify your configurations as a dict
    lgb_params = {
        
    #static parameters
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'feature_fraction': 0.9,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'verbose': 0,
    'learning_rate': 0.2,
    'max_depth': int(max_depth),
    'min_child_weight' : int(min_child_weight),
    'colsample_bytree' : max(min(colsample_bytree, 1), 0),
    'subsample' : max(min(subsample, 1), 0),
    'gamma' : max(gamma, 0), 
    }

    model = lgb.LGBMClassifier(**lgb_params)
    
    kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
    results = cross_val_score(model, data, target, cv = kfold)
    auc = results.mean()*100
    print("AUC : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    return auc

In [14]:
#lightGBM (parameters tuned with Bayesian Optimization) Result

lgb_params = {
    'max_depth' : (2, 10), 
    'min_child_weight' : (1, 10), 
    'colsample_bytree' : (0.1, 10), 
    'subsample' : (0.5, 1),
    'gamma' : (0, 10)
}


lgb_bayesOPT = BayesianOptimization(LGB_Train_Model_using_cv, lgb_params)
start_time = time.time()
lgb_bayesOPT.maximize(init_points = 5, n_iter = 100)
elapsed_time = time.time() - start_time
print("elpased time is %s"%elapsed_time)

Initialization
---------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
AUC : 89.30% (0.81%)
    1 | 00m00s |   89.29727 |             4.6545 |    8.3868 |      6.8908 |             3.2612 |      0.8925 | 
AUC : 89.30% (0.81%)
    2 | 00m00s |   89.29727 |             3.2188 |    4.1378 |      6.3477 |             8.6735 |      0.6477 | 
AUC : 87.56% (0.81%)
    3 | 00m00s |   87.56324 |             8.8533 |    6.3177 |      2.8810 |             3.4358 |      0.6382 | 
AUC : 88.87% (0.75%)
    4 | 00m00s |   88.87238 |             8.0273 |    9.2087 |      4.6613 |             3.9498 |      0.8818 | 
AUC : 89.03% (1.06%)
    5 | 00m00s |   89.03313 |             5.0754 |    1.0417 |      7.3226 |             2.0006 |      0.8785 | 
Bayesian Optimization
--------------------------------------------------------------------

In [37]:


import time

xgb_clf = xgb.XGBClassifier(eval_metric = 'auc', n_trees = 20)

xgb_params = {
    'learning_rate' : [0.2],
    'min_child_weight' : np.arange(1, 20, 5),      # 4
    'max_depth' : np.arange(2, 10, 2),             # 4 
    'gamma' : np.arange(0, 10, 2.5),                 # 4
    'subsample' : np.arange(0.5, 1.0, 0.25),        # 2
    'colsample_bytree' : np.arange(0.1, 1.0, 0.3), # 3
    'objective' : ['reg:linear'],
    'silent' : [1],
}

GSCV = GridSearchCV(xgb_clf, xgb_params, cv = 5, scoring = 'accuracy', n_jobs = 1, verbose = 1)

start_time = time.time()
GSCV.fit(data, target)
elapsed_time = time.time() - start_time
print("%s seconds elpased."%elapsed_time)




Fitting 5 folds for each of 384 candidates, totalling 1920 fits


[Parallel(n_jobs=1)]: Done 1920 out of 1920 | elapsed: 23.9min finished


1438.6167149543762 seconds elpased.


In [38]:
best_parameters, score, _ = max(GSCV.grid_scores_, key=lambda x: x[1])
print('AUC score:', score)
print('best parameters:', best_parameters)

AUC score: 0.8352090032154341
best parameters: {'colsample_bytree': 0.7000000000000001, 'gamma': 5.0, 'learning_rate': 0.2, 'max_depth': 8, 'min_child_weight': 16, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.75}


In [40]:

file_name1 = "../data/train_preprocessed1.csv"
file_name2 = "../data/train_preprocessed2.csv"

train_df1 = pd.read_csv(file_name1, low_memory = False)
train_df2 = pd.read_csv(file_name2, low_memory = False)

data_df1 = train_df1.drop(['Grant.Status'], axis = 1)
target_df1 = train_df1['Grant.Status']

data_df2 = train_df2.drop(['Grant.Status'], axis = 1)
target_df2 = train_df2['Grant.Status']

data1 = data_df1.values
target1 = target_df1.values
data2 = data_df2.values
target2 = target_df2.values


In [46]:

def preprocessing_XGB_KFold(preprocessing, min_child_weight, max_depth, gamma, subsample,  colsample_bytree) : 
    xgb_params = {
        'n_trees' : 20,
        'eta' : 0.2,
        'max_depth' : int(max_depth),
        'subsample' : max(min(subsample, 1), 0),
        'objective' : 'reg:linear', 
        'eval_metric' : 'auc',
        'silent' : 1,
        'min_child_weight' : int(min_child_weight),
        'gamma' : max(gamma, 0), 
        'colsample_bytree' : max(min(colsample_bytree, 1), 0)
    }
    preprocessing = round(preprocessing)
    if preprocessing == 1 : 
        data = data1
        target = target1
    else : 
        data = data2
        target = target2
    model = xgb.XGBClassifier(**xgb_params)
    
    kfold = KFold(n_splits = 5, random_state = 7)
    results = cross_val_score(model, data, target, cv = kfold)
    auc = results.mean()*100
    print("AUC : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    
    return auc

In [47]:
#XGB (parameters tuned by Bayesian Optimization) result

xgb_params = {
    'preprocessing' : (1, 2),
    'min_child_weight' : (1, 20), 
    'max_depth' : (2, 10),
    'gamma' : (0, 10),
    'subsample' : (0.5, 1),
    'colsample_bytree' : (0.1, 1)
}


xgb_bayesOPT = BayesianOptimization(preprocessing_XGB_KFold, xgb_params)
start_time = time.time()
xgb_bayesOPT.maximize(init_points = 5, n_iter = 100)
elapsed_time = time.time() - start_time
print("elapsed time : %s"%elapsed_time)

Initialization
---------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   preprocessing |   subsample | 
AUC : 81.49% (5.91%)
    1 | 00m12s |   81.48952 |             0.7895 |    8.5295 |      9.8107 |             5.0134 |          1.7871 |      0.5251 | 
AUC : 82.20% (5.98%)
    2 | 00m01s |   82.20467 |             0.2732 |    1.7262 |      4.6955 |            10.2691 |          1.0363 |      0.6589 | 
AUC : 84.96% (3.00%)
    3 | 00m06s |   84.95700 |             0.9594 |    0.3385 |      5.9580 |             1.2928 |          1.8937 |      0.7557 | 
AUC : 81.83% (5.75%)
    4 | 00m08s |   81.83394 |             0.7189 |    8.9734 |      7.8112 |            19.7673 |          1.9741 |      0.5979 | 
AUC : 80.30% (6.22%)
    5 | 00m01s |   80.29953 |             0.5373 |    8.3502 |      3.5183 |             4.5803

AUC : 85.58% (3.46%)
   48 | 00m14s |   85.57721 |             0.6671 |    2.9269 |      6.9001 |             3.2694 |          1.5832 |      0.9118 | 
AUC : 85.45% (3.58%)
   49 | 00m12s |   85.45090 |             0.3528 |    1.6198 |      5.7951 |             1.2725 |          2.0000 |      0.7994 | 
AUC : 82.17% (5.88%)
   50 | 00m14s |   82.16701 |             0.5660 |    6.7937 |      5.0224 |             4.9484 |          1.5012 |      0.7350 | 
AUC : 85.14% (3.34%)
   51 | 00m14s |   85.14082 |             0.5995 |    2.5486 |      6.7978 |             2.2762 |          1.8404 |      0.8801 | 
AUC : 81.93% (7.04%)
   52 | 00m13s |   81.92891 |             0.7416 |    3.9805 |      6.9019 |             2.5142 |          1.3407 |      1.0000 | 
AUC : 85.28% (3.31%)
   53 | 00m13s |   85.27858 |             0.4903 |    3.2451 |      6.3209 |             4.0432 |          1.9837 |      0.8335 | 
AUC : 81.94% (5.88%)
   54 | 00m19s |   81.93734 |             0.7452 |    7.7360 |     

AUC : 84.66% (3.59%)
  102 | 00m24s |   84.65868 |             0.7477 |    3.1513 |      5.4609 |             3.1625 |          1.8877 |      0.8340 | 
AUC : 85.24% (2.98%)
  103 | 00m25s |   85.24403 |             0.3771 |    1.8530 |      6.1530 |             1.4908 |          1.8888 |      0.8161 | 
AUC : 85.09% (3.07%)
  104 | 00m26s |   85.09483 |             0.5454 |    3.1790 |      7.0486 |             2.5516 |          1.6835 |      0.8414 | 
AUC : 84.70% (3.33%)
  105 | 00m28s |   84.70455 |             0.7973 |    3.0149 |      7.0695 |             3.5742 |          1.6017 |      0.8310 | 
elapsed time : 5927.012678861618
